# Курс Специализация Data Science Дипломный проект. Бриф учебного кейса 
## «Модель прогнозирования стоимости жилья для агентства недвижимости»

В данном ноутбуке мы сделаем следующее:
* Подгрузим обученную модель CatBoostRegressor
* Напишем функцию для предсказания стоимости недвижимости по входным параметрам
* Проведем тест на воспроизводимость
* Выведем нашу модель в продакшен


In [28]:
import numpy as np 
import pandas as pd 
import pickle
import warnings
warnings.filterwarnings("ignore")

In [29]:
# Загрузка сохраненной модели из файла pickle
with open("diplom_web/app/models/best_cb_model.pkl", "rb") as f:
    loaded_model = pickle.load(f)

In [30]:
# создадим тестовый набор 
data = [
    ('Active', 3.0, 'Washington', 801, 20004, 'DC', False, 'other', 1991, False, False, False, False, 6.0, 0.2),
    ('Active', 2.0, 'Dallas', 832, 75219, 'TX', False, 'condo', 1998, False, True, False, False, 2.6, 0.6),
    ('Active', 4.0, 'Dallas', 2102, 75219, 'TX', False, 'condo', 2019, False, True, False, False, 2.0, 0.1),
    ('Active', 2.0, 'Dallas', 837, 75219, 'TX', False, 'condo', 1998, False, True, False, False, 2.6, 0.6),
    ('Active', 2.0, 'Dallas', 2785, 75219, 'TX', False, 'condo', 2015, False, True, False, True, 2.6, 0.7)
]

columns = ['status', 'baths', 'city', 'sqft', 'zipcode', 'state', 'pool_encoded', 'Type', 'Year built', 'Heating_encoded', 'Cooling_encoded', 'Parking_encoded', 'fireplace_encoded', 'school_rating _mean', 'school_dist_min']

df_test = pd.DataFrame(data, columns=columns)

df_test.head()

,status,baths,city,sqft,zipcode,state,pool_encoded,Type,Year built,Heating_encoded,Cooling_encoded,Parking_encoded,fireplace_encoded,school_rating _mean,school_dist_min
0,Active,3.0,Washington,801,20004,DC,False,other,1991,False,False,False,False,6.0,0.2
1,Active,2.0,Dallas,832,75219,TX,False,condo,1998,False,True,False,False,2.6,0.6
2,Active,4.0,Dallas,2102,75219,TX,False,condo,2019,False,True,False,False,2.0,0.1
3,Active,2.0,Dallas,837,75219,TX,False,condo,1998,False,True,False,False,2.6,0.6
4,Active,2.0,Dallas,2785,75219,TX,False,condo,2015,False,True,False,True,2.6,0.7


In [32]:
# функция принимает датасет и модель, затем обрабатывет 
# датасет и передает его в модель для получения предсказания, предсказание выводится в нормальном виде
def preprocess_and_predict(df_input, model):
    
    def log_data(df_input):
        df_output = df_input.copy()
        df_output['zipcode'] = df_output['zipcode'].astype(str)
        df_output['Year built'] = df_output['Year built'].astype(str)
        
        #scaler = MinMaxScaler()
        for column in ['baths', 'sqft', 'school_rating _mean', 'school_dist_min']:
            #df_output[column] = scaler.fit_transform(df_output[[column]])[:,0]
            df_output[column] = df_output[column].apply(lambda x: abs(x))
            constant = 1e-6
            df_output[column] = np.log(df_output[column] + constant)
        return df_output

    X_test = log_data(df_input)
    y_test_pred_loaded = model.predict(X_test)
    target = np.exp(y_test_pred_loaded)
    rounded_target = np.round(target)
    print(rounded_target)
    
    return rounded_target

In [33]:
predictions = preprocess_and_predict(df_test, loaded_model)


[506389. 224225. 551250. 224081. 626743.]


### Модель воспроизводится!!!

In [ ]:
#pip freeze > requirements.txt

### Контейнеризация Docker

Образ, который будем использовать для своего контейнера — tiangolo/uwsgi-nginx-flask.  
Образ этот создан на базе ОС Linux, и в нём уже настроено взаимодействие серверов uWSGI и NGINX через сокеты.

Вся необходимая структура и сам конфигурационный файл для создания Docker образа находится в директории diplom_web.

Образ создаем в терминале ПК командой:

    docker build -t diplom .

Ждем окончания установки всех необходимых библиотек и запускаем нашь сервер командой
    
    docker run -p 5000:5000 diplom python server.py
![](image/Запуск.png)

после запуска страница браузера по адресу 127.0.0.1:5000 выдаст сообщение "Тестовое сообщение. Сервер запущен!"

далее нужно перейти в терминале в директорию test 

    cd test
    
и запустить скрипт отправки запроса client.py

    python3 client.py

![](image/Вывод.png)

Меняя в строке запроса (9) в файле client.py параметры предполагаемого объекта мы будем получать новое предсказание.

## Заключение 

На данном этапе мы имеем рабочий сервис с обученной моделью по поставленной нам задаче предсказания цены нового объекта недвижимости.